In [3]:
import sys
import os
import numpy as np
from torchvision import datasets, transforms

notebooks_dir = os.path.dirname(os.path.abspath("__file__")) 
project_root = os.path.dirname(notebooks_dir) 
scripts_path = os.path.join(project_root, "Scripts") 

sys.path.append(scripts_path)

from config import Config
from models import AutoEncoder
from utils import load_model
from dataset import get_dataloaders
import torch
import cv2
import matplotlib.pyplot as plt
from PIL import Image
config = Config()

transform = transforms.Compose([
        transforms.Resize((config.image_size, config.image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

In [4]:
model = AutoEncoder(config.latent_dim, config.image_size).to(config.device)
model.encoder.load_state_dict(torch.load(config.Encoder_path))
model.decoder.load_state_dict(torch.load(config.Decoder_path))

<All keys matched successfully>

In [7]:
def test():
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame_rgb)
        
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = transform(Image.fromarray(frame_rgb)).unsqueeze(0).to(config.device)
        with torch.no_grad():
            out, lat = model(image)
        recon =  out[0].permute(1,2,0).detach().cpu().numpy()
            
        
        
        cv2.imshow("Webcam Feed", recon)

        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [8]:
test()